In [124]:
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import ipynb.fs.defs.PeopleInfo as peopleInfo
from nltk.corpus import stopwords
from nltk import TweetTokenizer
# from nltk.stem.porter import PorterStemmer
# 引用 tweetTextHandler.ipynb
import ipynb.fs.defs.TweetTextHandler as tweetTextHandler
import ipynb.fs.defs.FilterMethods as filterMethods
import sys, os
sys.path.append('../2_feature')
import ipynb.fs.defs.GetFeatures as getFeatures
global stop_words
stop_words = stopwords.words('english')

In [2]:
def getWordsDict(texts):
    global stop_words
    wordDict = dict()
    stopwordDict = dict()
    stopwordTweetCount = 0
    tknzr = TweetTokenizer()

    for txt in texts:
        stopwordFlag = False
        try:
            txt = tweetTextHandler.del_url(txt)
            for token in tknzr.tokenize(txt):
                #token = porter.stem(token.lower())
                if token.isdigit():
                    pass
                elif token in stop_words:
                    stopwordDict[token] = stopwordDict.get(token, 0) + 1
                    stopwordFlag = True
                else:
                    wordDict[token] = wordDict.get(token, 0) + 1
            if stopwordFlag:
                stopwordTweetCount += 1
        except Exception as e:
            #print("Error message:", e)
            pass
    return wordDict, stopwordDict, stopwordTweetCount

In [3]:
patients = dict()
ordinarys = dict()

In [4]:
with open('../0_dataset/patient_ids') as r:
    for patient in r.readlines()[:100]:
        patient = patient.strip()
        patients[patient] = peopleInfo.Patient(patient)

In [5]:
patients = filterMethods.by_tweets_number(patients)

Remove users:['181095972', '2409775369', '363595479', '2382824556', '113666708', '2195175613', '402683346', '3031516391', '1229103648']


In [6]:
with open('../0_dataset/ordinary_ids') as r:
    for ordinary in r.readlines()[:100]:
        ordinary = ordinary.strip()
        ordinarys[ordinary] = peopleInfo.Ordinary(ordinary)

In [7]:
ordinarys = filterMethods.by_tweets_number(ordinarys)

Remove users:[]


In [8]:
import plotly
# import plotly.offline as offline
import plotly.plotly as py
import plotly.graph_objs as go
# from IPython.display import display
plotly.tools.set_credentials_file(username='Adeline', api_key='Z5eltNtBQXqvI05ZFQtz')
# offline.init_notebook_mode(connected=True)

In [9]:
total3 = [0, 0, 0]
for key in patients:
    _w, _s, _sc = getWordsDict(patients[key].getText())
    total3[0] += sum(_s.values())
    total3[1] += _sc
    total3[2] += len(patients[key].inRangeDf)
    
# 0:Stopwords總數/ 1:含有stopwords的推文總數/ 2:（時間內的）推文總數

In [10]:
total4 = [0, 0, 0]
for key in ordinarys:
    _w, _s, _sc = getWordsDict(ordinarys[key].getText())
    total4[0] += sum(_s.values())
    total4[1] += _sc
    total4[2] += len(ordinarys[key].inRangeDf)

# 0:Stopwords總數/ 1:含有stopwords的推文總數/ 2:（時間內的）推文總數

In [11]:
trace_patients = go.Bar(x=['Type1', 'Type2'],
                  y=[round(total3[0]/total3[1],1), round(total3[1]/total3[2],1)],
                  name='Patients',
                  marker=dict(color='#ffcdd2'))

trace_ordinarys = go.Bar(x=['Type1', 'Type2'],
                y=[round(total4[0]/total4[1],1), round(total4[1]/total4[2],1)],
                name='Ordinarys',
                marker=dict(color='#A2D5F2'))
data = [trace_patients, trace_ordinarys]

layout = go.Layout(title='比較',
                xaxis=dict(title='使用者類型'),
                yaxis=dict(title='平均數量'))

fig3 = go.Figure(data=data, layout=layout)
py.image.save_as(fig3, filename='img/a-simple-plot.png')

In [138]:
def filter_tweet(txt):
    
    try:
        # url
        txt = re.sub(r"http\S+", "", txt)
        txt = re.sub(r"https\S+", "", txt)
        txt = re.sub(r"twitter.com/\S+", "", txt)
        txt = re.sub(r"pic.twitter.com\S+", "", txt)
        # mention
        txt = re.sub(r"@\S+", "", txt)
    except TypeError as e:
        print(e)
    
    return txt

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [139]:
base_texts = []
group_texts = []

for key in ordinarys.keys():
    base_texts.append(filter_tweet('\n'.join(ordinarys[key].getText())))

for key in patients.keys():
    group_texts.append(filter_tweet('\n'.join(patients[key].getText())))

corpus = base_texts + group_texts

In [126]:
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_
Y = np.array([0] * len(base_texts) + [1]*len(group_texts), dtype=int)
classifier  = RandomForestClassifier(n_jobs=-1, max_features="sqrt", n_estimators=128)
classifier.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [127]:
feature_indicies = np.argsort(classifier.feature_importances_)
vector2word = vectorizer.get_feature_names()

In [128]:
top_words = []
for i in range(50):
    vector = feature_indicies[-i-1]
    word = vector2word[vector]
    top_words.append(word)

In [129]:
top_words

['depression',
 'pic',
 'tus',
 'wtf',
 'status',
 'diagnosed',
 'wrong',
 'com',
 'bitch',
 'pick',
 'heart',
 'care',
 'piece',
 'tatus',
 'cry',
 'falling',
 'anxiety',
 'okay',
 'sorry',
 'into',
 'their',
 'goal',
 'never',
 'read',
 'cried',
 'myself',
 'happened',
 'they',
 'am',
 'atus',
 'person',
 'finally',
 'idk',
 'old',
 'and',
 'tho',
 'love',
 'watched',
 'said',
 'alone',
 'friends',
 'felt',
 'deserve',
 'please',
 'you',
 'close',
 'kind',
 'la',
 'easier',
 'favorite']

In [141]:
group_texts

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
